# Energy Carbon: Marketshare by Country Weighting
In this first section we will determine the country-wise countribution to each of the energy steps of the silicon supply chain to correctly attribute carbon intensities to grids.

In [17]:
import numpy as np
import pandas as pd
import os,sys
from pathlib import Path
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (10, 6)
cwd = os.getcwd() #grabs current working directory
carbonfolder = str(Path().resolve().parent.parent.parent / 'PV_ICE' / 'baselines' / 'SupportingMaterial' / 'CarbonIntensities')
supportmatfolder = str(Path().resolve().parent.parent.parent / 'PV_ICE' / 'baselines' / 'SupportingMaterial')

Pull in crunched data from BNEF commissioned capacity to create country marketshare. Everything that was less than 1% of global marketshare was excluded (things don't currently sum to 100). 

In [15]:
#skipcols = ['Source', 'Notes','Country']
mrktshare_country_si_cell = pd.read_csv(os.path.join(carbonfolder, 'input-silicon-CountryMarketshare-cell.csv'),
                                     index_col='Country')#, usecols=lambda x: x not in skipcols)
mrktshare_country_si_wafer = pd.read_csv(os.path.join(carbonfolder, 'input-silicon-CountryMarketshare-wafer.csv'),
                                     index_col='Country')#, usecols=lambda x: x not in skipcols)
mrktshare_country_si_ingot_multi = pd.read_csv(os.path.join(carbonfolder, 'input-silicon-CountryMarketshare-ingot-multi.csv'),
                                     index_col='Country')#, usecols=lambda x: x not in skipcols)
mrktshare_country_si_ingot_mono = pd.read_csv(os.path.join(carbonfolder, 'input-silicon-CountryMarketshare-ingot-mono.csv'),
                                     index_col='Country')#, usecols=lambda x: x not in skipcols)
mrktshare_country_si_polysi = pd.read_csv(os.path.join(carbonfolder, 'input-silicon-CountryMarketshare-polysilicon.csv'),
                                     index_col='Country')#, usecols=lambda x: x not in skipcols)
#mrktshare_country_si_silica = pd.read_csv(os.path.join(carbonfolder, 'input-silicon-CountryMarketshare-silica.csv'),
#                                     index_col='Country')#, usecols=lambda x: x not in skipcols)


Pull in silicon energy files to create a % of energy attributable to each step of the MFGing process. This will be multiplied by the country market shares to create an overall country marketshare of the energy supply chain (i.e. energy weighted...)

Also the mono vs multi to attribute marketshare of these two technologies.

In [52]:
energy_silicon_mfg = pd.read_csv(os.path.join(supportmatfolder, 'output_energy_silicon_mfg.csv'),
                                     index_col=0)#, usecols=lambda x: x not in skipcols)
marketshare_silicon_monoVmulti = pd.read_csv(os.path.join(supportmatfolder, 'output_scaledmrktshr_mcSi_mono.csv'),
                                     index_col=0)#, usecols=lambda x: x not in skipcols)
energy_silicon_mfg_STEPS_raw = pd.read_csv(os.path.join(supportmatfolder, 'output_energy_silicon_mfg_STEPS.csv'),
                                     index_col=0)#, usecols=lambda x: x not in skipcols)

In [53]:
energy_silicon_mfg_STEPS = energy_silicon_mfg_STEPS_raw.filter(like="kWhpkg") #drop the fuel fraction columns
energy_silicon_mfg.drop(['E_mfgFuelFraction'], axis=1, inplace=True) #drop fuel fraction


In [73]:
energyShare_silicon_mfg_STEPS = energy_silicon_mfg_STEPS.div(energy_silicon_mfg.values)
#sanity check that everything adds to 100% or 1
#energyShare_silicon_mfg_STEPS.sum(axis=1)

In [77]:
energyShare_silicon_mfg_STEPS.columns

Index(['E_reduce_sum_kWhpkg', 'E_refineSiemens_kWhpkg', 'E_Ingot_kWhpkg',
       'E_Wafering_kWhpkg', 'E_cellProcess_kWhpkg'],
      dtype='object')

The math is:

    % of energy for mfging step * % country contribution, then sum for each country across all energy steps to get the country contribution to MFGing energy

In [83]:
energyShare_silicon_mfg_STEPS_subset = energyShare_silicon_mfg_STEPS.loc[2004:2022]
energyShare_silicon_mfg_STEPS_subset

,E_reduce_sum_kWhpkg,E_refineSiemens_kWhpkg,E_Ingot_kWhpkg,E_Wafering_kWhpkg,E_cellProcess_kWhpkg
2004,0.157930,0.585792,0.085928,0.075825,0.094546
2005,0.154927,0.550520,0.178615,0.034834,0.081090
2006,0.161192,0.562690,0.163139,0.036978,0.075986
2007,0.158861,0.547185,0.188248,0.037345,0.068346
2008,0.156184,0.530331,0.213765,0.037587,0.062152
2009,0.156868,0.525829,0.219577,0.038694,0.059017
2010,0.158375,0.525649,0.217818,0.040153,0.058024
2011,0.164488,0.541360,0.190611,0.042912,0.060642
2012,0.163888,0.535220,0.193970,0.044008,0.062907
2013,0.166651,0.540544,0.177866,0.046080,0.068841


### Cell

In [113]:
energy_cell_fract = energyShare_silicon_mfg_STEPS_subset['E_cellProcess_kWhpkg']

In [106]:
mrktshare_country_si_cell_fractbyyear = mrktshare_country_si_cell/100 #turn it into a decimal

In [112]:
cell_by_country = mrktshare_country_si_cell_fractbyyear*energy_cell_fract.values*100
# sanity check: that all countries each year add to the value from energy_cell_fract or pretty close
#cell_by_country.sum()/100

### Wafer

In [141]:
energy_wafer_fract = energyShare_silicon_mfg_STEPS_subset['E_Wafering_kWhpkg']
#energy_wafer_fract

In [142]:
mrktshare_country_si_wafer_fractbyyear = mrktshare_country_si_wafer/100 #turn it into a decimal
#mrktshare_country_si_wafer_fractbyyear

In [143]:
wafer_by_country = mrktshare_country_si_wafer_fractbyyear*energy_wafer_fract.values*100
# sanity check: that all countries each year add to the value from energy_cell_fract or pretty close
#wafer_by_country.sum()/100
#wafer_by_country

### Ingot
More complicated due to mono vs multi marketshare. The total ingot energy is already weighted by this marketshare. Math is:
        
        (mono% * % mono country production + multi% * % multi country production)* Ingot % of energy.

In [133]:
marketshare_silicon_monoVmulti_subset = marketshare_silicon_monoVmulti.loc[2004:2022] #in fraction form
marketshare_silicon_monoVmulti_subset.head()

,monoSi,mcSi
Year,,
2004,0.080000,0.920000
2005,0.309000,0.691000
2006,0.270000,0.730000
2007,0.353535,0.646465
2008,0.454545,0.545455


In [134]:
energy_ingot_fract = energyShare_silicon_mfg_STEPS_subset['E_Ingot_kWhpkg']
energy_ingot_fract

2004    0.085928
2005    0.178615
2006    0.163139
2007    0.188248
2008    0.213765
2009    0.219577
2010    0.217818
2011    0.190611
2012    0.193970
2013    0.177866
2014    0.175012
2015    0.182446
2016    0.162621
2017    0.147205
2018    0.155731
2019    0.158004
2020    0.203357
2021    0.211166
2022    0.212197
Name: E_Ingot_kWhpkg, dtype: float64

In [136]:
mrktshare_country_si_ingot_multi_fractbyyear = mrktshare_country_si_ingot_multi/100 #turn it into a decimal
mrktshare_country_si_ingot_mono_fractbyyear = mrktshare_country_si_ingot_mono/100 #turn it into a decimal

In [152]:
#mono multi deployment marketshares * the countrywise marketshare of mfging these techs
country_multi = mrktshare_country_si_ingot_multi_fractbyyear*marketshare_silicon_monoVmulti_subset['mcSi'].values
country_mono = mrktshare_country_si_ingot_mono_fractbyyear*marketshare_silicon_monoVmulti_subset['monoSi'].values

In [ ]:
#So there are not the same countries making these technologies

In [159]:
#weight both by the ingot energy
country_multi_energy = country_multi*energy_ingot_fract.values
country_mono_energy = country_mono*energy_ingot_fract.values

In [161]:
country_multi_energy.index

Index(['China (mainland)', 'Germany', 'India', 'Korea (Republic)', 'Taiwan',
       'United Kingdom'],
      dtype='object', name='Country')

In [162]:
country_mono_energy.index

Index(['China (mainland)', 'Germany', 'Korea (Republic)', 'Malaysia', 'Norway',
       'Qatar', 'Taiwan', 'Ukraine', 'United States', 'Vietnam'],
      dtype='object', name='Country')

In [180]:
country_wtdenergy_ingot = pd.concat([country_mono_energy, country_multi_energy]).groupby(['Country']).sum()
#sanity check that sums by year are the same as the ingot energy fraction
#country_wtdenergy_ingot.sum(axis=0)

In [181]:
ingot_by_country = country_wtdenergy_ingot*100

### Polysilicon

In [126]:
energy_polysi_fract = energyShare_silicon_mfg_STEPS_subset['E_refineSiemens_kWhpkg']
#energy_polysi_fract

In [127]:
mrktshare_country_si_polysi_fractbyyear = mrktshare_country_si_polysi/100 #turn it into a decimal

In [128]:
polysi_by_country = mrktshare_country_si_polysi_fractbyyear*energy_polysi_fract.values*100
# sanity check: that all countries each year add to the value from energy_cell_fract or pretty close
#polysi_by_country.sum()/100

### Silica

In [120]:
energy_reducesilica_fract = energyShare_silicon_mfg_STEPS_subset['E_reduce_sum_kWhpkg']
#energy_wafer_fract

In [121]:
mrktshare_country_si_silica_fractbyyear = mrktshare_country_si_silica/100 #turn it into a decimal

In [122]:
silicareduce_by_country = mrktshare_country_si_silica_fractbyyear*energy_reducesilica_fract.values*100
# sanity check: that all countries each year add to the value from energy_cell_fract or pretty close
#wafer_by_country.sum()/100